In [142]:
from bs4 import BeautifulSoup as bs
import re 
import lxml
import xml.etree.ElementTree as ET

### Method 1 (skrottet)

In [225]:
from lxml import objectify

file = "20091_M1_helemoedet.xml"

with open(file) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

In [87]:
root.attrib

{'edixistatus': ''}

### Read xml file 

In [201]:
tree = ET.parse('20091_M100_helemoedet.xml')
root = tree.getroot()

In [169]:
root[0].tag
root[1].tag

'TitelGruppe'

##### Subelements / children

In [161]:
# children of the root only 
root_children = []

for child in root:
    root_children.append((child.tag, child.attrib))

In [160]:
# All children in file
# d = [elem.tag for elem in root.iter()]

In [164]:
# entire xml file as string
xml = ET.tostring(root, encoding='utf8').decode('utf8')

### Method 3: soup

In [133]:
soup = bs(b, 'lxml')

In [247]:
dagsordenpunkter = soup.find_all("dagsordenpunkt") # punkterne med tags 
punkter = []  # punkterne uden tags 

for dagsordenpunkt in range(0, len(dagsordenpunkter)):
    punkter.append(result[dagsordenpunkt].get_text(separator = ' '))